In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

### Defining variables


In [2]:
h = np.array([1, -3, 3, -1])
t = 2
q = 3
n = 4

window_size = (128, 128)
step_size = 8

### Loading video


In [3]:
video = cv2.VideoCapture('videos/original(Video-2).avi')

width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print('Video shape: {}x{}x{}x{}'.format(width, height, fps, frames))

forgedVideo = cv2.VideoCapture('videos/forged(Video-2)-1.avi')

fwidth = int(forgedVideo.get(cv2.CAP_PROP_FRAME_WIDTH))
fheight = int(forgedVideo.get(cv2.CAP_PROP_FRAME_HEIGHT))
ffps = int(forgedVideo.get(cv2.CAP_PROP_FPS))
fframes = int(forgedVideo.get(cv2.CAP_PROP_FRAME_COUNT))
print('Forged video shape: {}x{}x{}x{}'.format(fwidth, fheight, ffps, fframes))

Video shape: 768x576x30x390
Forged video shape: 768x576x25x334


### Play the video


In [ ]:
while True:
  ret, frame = video.read()
  if not ret:
    break
  cv2.imshow('frame', frame)
  if cv2.waitKey(1000//fps) & 0xFF == ord('q'):
    break

cv2.destroyAllWindows()
video.set(cv2.CAP_PROP_POS_FRAMES, 0)

### Play forged video


In [8]:
while True:
  ret, frame = forgedVideo.read()
  if not ret:
    break
  cv2.imshow('frame', frame)
  if cv2.waitKey(1000//fps) & 0xFF == ord('q'):
    break

cv2.destroyAllWindows()
forgedVideo.set(cv2.CAP_PROP_POS_FRAMES, 0)

True

### Storing the video frames in an array


In [4]:
videoFrames = []
while True:
  ret, frame = video.read()
  if not ret:
    break
  videoFrames.append(frame)

videoFrames = np.array(videoFrames)
print(videoFrames.shape)

fvideoFrames = []
while True:
  ret, frame = forgedVideo.read()
  if not ret:
    break
  fvideoFrames.append(frame)

fvideoFrames = np.array(fvideoFrames)
print(fvideoFrames.shape)

(390, 576, 768, 3)
(334, 576, 768, 3)


### Play video using videoFrames


In [ ]:
for i in range(videoFrames.shape[0]):
  print(i)
  cv2.imshow('frame', videoFrames[i, :, :, :])
  if cv2.waitKey(1000//fps) & 0xFF == ord('q'):
    break
    
cv2.destroyAllWindows()
video.set(cv2.CAP_PROP_POS_FRAMES, 0)

### Co-occurence matrix


In [5]:
def cooc_matrix_row(residual):
  co_occurrence_matrix = np.zeros((1,625))
  for i in range(residual.shape[0]):
    for j in range(residual.shape[1]-4):
      key1 = (2+residual[i,j])*125 + (2+residual[i,j+1])*25 + (2+residual[i,j+2])*5 + (2+residual[i,j+3])
      key2 = (2+residual[i,j+3])*125 + (2+residual[i,j+2])*25 + (2+residual[i,j+1])*5 + (2+residual[i,j])
      key = min(key1, key2).astype(int)
      co_occurrence_matrix[0,key] += 1
  return co_occurrence_matrix

def cooc_matrix_col(residual):
  co_occurrence_matrix = np.zeros((1,625))
  for i in range(residual.shape[0]-4):
    for j in range(residual.shape[1]):
      key1 = (2+residual[i,j])*125 + (2+residual[i+1,j])*25 + (2+residual[i+2,j])*5 + (2+residual[i+3,j])
      key2 = (2+residual[i+3,j])*125 + (2+residual[i+2,j])*25 + (2+residual[i+1,j])*5 + (2+residual[i,j])
      key = min(key1, key2).astype(int)
      co_occurrence_matrix[0,key] += 1
  return co_occurrence_matrix

def cooc_matrix_tem(residual):
  cooc_temp = []
  for i in range(residual.shape[0]-4):
    co_occurrence_matrix = np.zeros((1,625))
    for j in range(residual.shape[1]):
      for k in range(residual.shape[2]):
        key1 = (2+residual[i,j,k])*125 + (2+residual[i+1,j,k])*25 + (2+residual[i+2,j,k])*5 + (2+residual[i+3,j,k])
        key2 = (2+residual[i+3,j,k])*125 + (2+residual[i+2,j,k])*25 + (2+residual[i+1,j,k])*5 + (2+residual[i,j,k])
        key = min(key1, key2).astype(int)
        co_occurrence_matrix[0,key] += 1
    cooc_temp.append(co_occurrence_matrix)
  co_occurrence_matrix = np.zeros((1,625))
  for i in range(4):
    cooc_temp.append(co_occurrence_matrix)
  return cooc_temp

### compute_descriptor()


In [6]:
def compute_descriptor_row(residual):
  cooc = []
  for i in range(residual.shape[0]):
    cooc.append(cooc_matrix_row(residual[i]))
  cooc = np.array(cooc)
  print("Row co-occurrence computed")
  return cooc

def compute_descriptor_col(residual):
  cooc = []
  for i in range(residual.shape[0]):
    cooc.append(cooc_matrix_col(residual[i]))
  cooc = np.array(cooc)
  print("Col co-occurrence computed")
  return cooc

def compute_descriptor_tem(residual):
  cooc_tem = cooc_matrix_tem(residual)
  cooc_tem = np.array(cooc_tem)
  print("Temporal co-occurrence computed")
  return cooc_tem

def compute_residuals(videoFrames):
  videoGrayScale = np.zeros((videoFrames.shape[0], videoFrames.shape[1], videoFrames.shape[2]))
  for i in range(videoFrames.shape[0]):
    videoGrayScale[i] = cv2.cvtColor(videoFrames[i], cv2.COLOR_BGR2GRAY)
  print("Video converted to grascale")
  
  row_resid = np.apply_along_axis(lambda x: np.convolve(x, h, mode='same'), 2, videoGrayScale)
  col_resid = np.apply_along_axis(lambda x: np.convolve(x, h, mode='same'), 1, videoGrayScale)
  temp_resid = np.apply_along_axis(lambda x: np.convolve(x, h, mode='same'), 0, videoGrayScale)

  row_resid = np.minimum(t, np.maximum(-t, np.floor(row_resid/q)))
  col_resid = np.minimum(t, np.maximum(-t, np.floor(col_resid/q)))
  temp_resid = np.minimum(t, np.maximum(-t, np.floor(temp_resid/q)))
  print("Residual computed")

  return row_resid, col_resid, temp_resid

### Finding residual of whole video


In [7]:
resids = compute_residuals(videoFrames)

Video converted to grascale


In [ ]:
cooc_row = compute_descriptor_row(resids[0])

In [ ]:
cooc_col = compute_descriptor_col(resids[1])

In [ ]:
cooc_tem = compute_descriptor_tem(resids[2])

In [8]:
forged_resids = compute_residuals(fvideoFrames)
forged_cooc_row = compute_descriptor_row(forged_resids[0])
forged_cooc_col = compute_descriptor_col(forged_resids[1])
forged_cooc_tem = compute_descriptor_tem(forged_resids[2])

Video converted to grascale
Residual computed
Row co-occurrence computed
Col co-occurrence computed
Temporal co-occurrence computed


In [ ]:
coocc_matrix = np.concatenate((cooc_row, cooc_col, cooc_tem), axis=1)
# coocc_matrix = np.reshape(coocc_matrix, (313, 3, 625))
print(coocc_matrix.shape)

In [10]:
forged_cooc_mat = np.concatenate((forged_cooc_row, forged_cooc_col, forged_cooc_tem), axis=1)
# forged_cooc_mat = np.reshape(forged_cooc_mat, (313, 3, 625))
print(forged_cooc_mat.shape)

(313, 3, 625)


### Mahalanobis distance

In [11]:
def findMean(descriptors):
  sum = np.apply_along_axis(lambda x: np.sum(x), 0, descriptors)
  mean = sum/descriptors.shape[0]
  return mean

def findCovariance(descriptors, mean):
  covariance = np.zeros((3, 3))
  for i in range(descriptors.shape[0]):
    covariance += np.matmul((descriptors[i]-mean), (descriptors[i]-mean).T)
  covariance /= descriptors.shape[0]
  return covariance

def findMahalanobisDistance(descriptors, mean, covariance):
  mahalanobis_distance = np.zeros((descriptors.shape[0], 625, 625))
  for i in range(descriptors.shape[0]):
    mahalanobis_distance[i] = np.sqrt(np.abs(np.matmul(np.matmul((descriptors[i]-mean).T, np.linalg.inv(covariance)), (descriptors[i]-mean))))
  return mahalanobis_distance

In [12]:
mean = findMean(coocc_matrix)
print("Mean : {}".format(mean.shape))
covariance = findCovariance(coocc_matrix, mean)
print("Covariance : {}".format(covariance.shape))
mahalanobis_dist = findMahalanobisDistance(coocc_matrix, mean, covariance)
print("Mahalanobis Distance : {}".format(mahalanobis_dist.shape))
forged_maha_dist = findMahalanobisDistance(forged_cooc_mat, mean, covariance)
print("Forged Mahalanobis Distance : {}".format(forged_maha_dist.shape))

Mean : (3, 625)
Covariance : (3, 3)
Mahalanobis Distance : (313, 625, 625)
Forged Mahalanobis Distance : (313, 625, 625)


In [13]:
diff = np.abs(mahalanobis_dist - forged_maha_dist)
print(diff.shape)

(313, 625, 625)


In [15]:
print(resids[0].shape)

(313, 240, 320)


In [51]:
max_dist = np.max(diff[290])
print(max_dist)

0.14747940719222627


In [ ]:
for i in range(227, 313):
  print("Frame : {}".format(i))
  cv2.imshow("Original", videoFrames[i])
  cv2.imshow("Forged", fvideoFrames[i])
  if cv2.waitKey(0) & 0xFF == ord('q'):
    break
  
cv2.destroyAllWindows()

In [62]:
start = 0
for i in range(diff.shape[0]):
  if np.max(diff[i]) > 0:
    print(i, np.max(diff[i]))

227 0.007526474494322707
228 0.03757291966411663
229 0.059131904571378895
230 0.06742122996661984
231 0.10167583481800245
232 0.19912355317987365
233 0.39972682005198423
234 0.33375899152994376
235 0.458925507594269
236 0.5655428529194899
237 0.43669445941966284
238 0.39663752257504503
239 0.2996999296411638
240 0.6326302235266361
241 1.0097755589350172
242 1.2415536189106835
243 1.4817450327865935
244 1.241650540494335
245 0.7852302618787892
246 1.0173464791376774
247 0.7161077961080983
248 1.0404571840457946
249 0.9165493845297124
250 1.0108232845611482
251 0.9779816734634458
252 0.862727849070501
253 0.3611532007765331
254 0.6103204317499564
255 0.9895589191759165
256 0.8130126398717583
257 0.5570919852879506
258 0.18973098373464575
259 0.2078290527818672
260 0.2542297086749614
261 0.25319414598980206
262 0.21676125232516574
263 0.2868987835130467
264 0.31916128080095857
265 0.3538889698564535
266 0.3094299563865389
267 0.3082667564673973
268 0.347179038474143
269 0.368385265102286


In [58]:
# get position of max value in diff[240]
max_pos = np.unravel_index(np.argmax(diff[241], axis=None), diff[241].shape)
print(max_pos)

(104, 104)


In [59]:
print(diff[241][max_pos[0]][max_pos[1]])

1.0097755589350172


In [ ]:
def generateHeatMap(frame1, frame2, fr_num):
  new_frame = np.zeros((frame1.shape[0], frame1.shape[1]-4))
  for i in range(frame1.shape[0]):
    for j in range(frame1.shape[1]-4):
      val1 = frame1[i,j:j+4]
      val2 = frame2[i,j:j+4]
      key1 = (2+val1[0])*125 + (2+val1[1])*25 + (2+val1[2])*5 + (2+val1[3])
      key2 = (2+val1[3])*125 + (2+val1[2])*25 + (2+val1[1])*5 + (2+val1[0])
      k1 = int(min(key1, key2))
      key1 = (2+val2[0])*125 + (2+val2[1])*25 + (2+val2[2])*5 + (2+val2[3])
      key2 = (2+val2[3])*125 + (2+val2[2])*25 + (2+val2[1])*5 + (2+val2[0])
      k2 = int(min(key1, key2))
      dist = diff[fr_num][k1][k2]
      dist = int(dist*255/1.241650540494335)
      new_frame[i][j] = dist
  return new_frame

newframe = generateHeatMap(resids[0][244], resids[0][244], 244)

In [67]:
print(newframe)

[[  3.   0.   4. ...   2.  13.   4.]
 [255. 255.   2. ...  48.  48.  12.]
 [  8.   1.   0. ...  27.  49.  43.]
 ...
 [  8.   1.   8. ...  43.  48. 255.]
 [  8.   0.   1. ...   0.   4.   7.]
 [ 13.  14.   0. ...   1.   0.   2.]]


In [11]:
cv2.imshow("frame", newframe)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'newframe' is not defined

### Finding residual step wise of a windows


In [89]:
broke = False
for f in range(videoFrames.shape[0]):
  print(f)
  for i in range(num_winw):
    for j in range(num_winh):
      # window = videoFrames[f, j*step_size:j*step_size+window_size[0], i*step_size:i*step_size+window_size[1], :]
      window = videoFrames[f, :, :, :]
      try:
        descriptor = compute_descriptor(window)
      except Exception as error:
        descriptor = True
        print(error)
        print("Error in frame {}, window {}, {}".format(f, i, j))
      if descriptor:
        broke = True
        break
    if broke:
      break
  if broke:
    break
cv2.destroyAllWindows()

0


### Destroy windows


In [31]:
cv2.destroyAllWindows()